# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f15f8c1ff70>
├── 'a' --> tensor([[ 0.0998, -0.5788,  0.4494],
│                   [ 1.8641,  0.5057,  0.6929]])
└── 'x' --> <FastTreeValue 0x7f15f8c78df0>
    └── 'c' --> tensor([[ 0.5706,  0.7665, -0.9097, -1.1767],
                        [ 0.4324,  0.6304, -1.9714, -0.8295],
                        [-0.2734,  1.0736, -2.5486, -0.1443]])

In [4]:
t.a

tensor([[ 0.0998, -0.5788,  0.4494],
        [ 1.8641,  0.5057,  0.6929]])

In [5]:
%timeit t.a

77.2 ns ± 1.61 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f15f8c1ff70>
├── 'a' --> tensor([[ 0.2423,  2.5487,  0.1126],
│                   [ 0.3788,  0.5232, -0.3321]])
└── 'x' --> <FastTreeValue 0x7f15f8c78df0>
    └── 'c' --> tensor([[ 0.5706,  0.7665, -0.9097, -1.1767],
                        [ 0.4324,  0.6304, -1.9714, -0.8295],
                        [-0.2734,  1.0736, -2.5486, -0.1443]])

In [7]:
%timeit t.a = new_value

77.6 ns ± 0.222 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.0998, -0.5788,  0.4494],
               [ 1.8641,  0.5057,  0.6929]]),
    x: Batch(
           c: tensor([[ 0.5706,  0.7665, -0.9097, -1.1767],
                      [ 0.4324,  0.6304, -1.9714, -0.8295],
                      [-0.2734,  1.0736, -2.5486, -0.1443]]),
       ),
)

In [10]:
b.a

tensor([[ 0.0998, -0.5788,  0.4494],
        [ 1.8641,  0.5057,  0.6929]])

In [11]:
%timeit b.a

71.9 ns ± 0.206 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-2.8043,  0.0860,  0.5400],
               [-0.4397, -1.2389, -0.0896]]),
    x: Batch(
           c: tensor([[ 0.5706,  0.7665, -0.9097, -1.1767],
                      [ 0.4324,  0.6304, -1.9714, -0.8295],
                      [-0.2734,  1.0736, -2.5486, -0.1443]]),
       ),
)

In [13]:
%timeit b.a = new_value

578 ns ± 1.91 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

938 ns ± 3.68 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.2 µs ± 52.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

166 µs ± 1.03 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

164 µs ± 421 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f154e9d5370>
├── 'a' --> tensor([[[ 0.0998, -0.5788,  0.4494],
│                    [ 1.8641,  0.5057,  0.6929]],
│           
│                   [[ 0.0998, -0.5788,  0.4494],
│                    [ 1.8641,  0.5057,  0.6929]],
│           
│                   [[ 0.0998, -0.5788,  0.4494],
│                    [ 1.8641,  0.5057,  0.6929]],
│           
│                   [[ 0.0998, -0.5788,  0.4494],
│                    [ 1.8641,  0.5057,  0.6929]],
│           
│                   [[ 0.0998, -0.5788,  0.4494],
│                    [ 1.8641,  0.5057,  0.6929]],
│           
│                   [[ 0.0998, -0.5788,  0.4494],
│                    [ 1.8641,  0.5057,  0.6929]],
│           
│                   [[ 0.0998, -0.5788,  0.4494],
│                    [ 1.8641,  0.5057,  0.6929]],
│           
│                   [[ 0.0998, -0.5788,  0.4494],
│                    [ 1.8641,  0.5057,  0.6929]]])
└── 'x' --> <FastTreeValue 0x7f154e9d5490>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

37 µs ± 120 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f154e8ffd30>
├── 'a' --> tensor([[ 0.0998, -0.5788,  0.4494],
│                   [ 1.8641,  0.5057,  0.6929],
│                   [ 0.0998, -0.5788,  0.4494],
│                   [ 1.8641,  0.5057,  0.6929],
│                   [ 0.0998, -0.5788,  0.4494],
│                   [ 1.8641,  0.5057,  0.6929],
│                   [ 0.0998, -0.5788,  0.4494],
│                   [ 1.8641,  0.5057,  0.6929],
│                   [ 0.0998, -0.5788,  0.4494],
│                   [ 1.8641,  0.5057,  0.6929],
│                   [ 0.0998, -0.5788,  0.4494],
│                   [ 1.8641,  0.5057,  0.6929],
│                   [ 0.0998, -0.5788,  0.4494],
│                   [ 1.8641,  0.5057,  0.6929],
│                   [ 0.0998, -0.5788,  0.4494],
│                   [ 1.8641,  0.5057,  0.6929]])
└── 'x' --> <FastTreeValue 0x7f154e8fffa0>
    └── 'c' --> tensor([[ 0.5706,  0.7665, -0.9097, -1.1767],
                        [ 0.4324,  0.6304, -1.9714, -0.8295],
                 

In [23]:
%timeit t_cat(trees)

34.4 µs ± 446 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

64.3 µs ± 465 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.5706,  0.7665, -0.9097, -1.1767],
                       [ 0.4324,  0.6304, -1.9714, -0.8295],
                       [-0.2734,  1.0736, -2.5486, -0.1443]],
              
                      [[ 0.5706,  0.7665, -0.9097, -1.1767],
                       [ 0.4324,  0.6304, -1.9714, -0.8295],
                       [-0.2734,  1.0736, -2.5486, -0.1443]],
              
                      [[ 0.5706,  0.7665, -0.9097, -1.1767],
                       [ 0.4324,  0.6304, -1.9714, -0.8295],
                       [-0.2734,  1.0736, -2.5486, -0.1443]],
              
                      [[ 0.5706,  0.7665, -0.9097, -1.1767],
                       [ 0.4324,  0.6304, -1.9714, -0.8295],
                       [-0.2734,  1.0736, -2.5486, -0.1443]],
              
                      [[ 0.5706,  0.7665, -0.9097, -1.1767],
                       [ 0.4324,  0.6304, -1.9714, -0.8295],
                       [-0.2734,  1.0736, -2.5486, -0.1443]],

In [26]:
%timeit Batch.stack(batches)

90.2 µs ± 530 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.5706,  0.7665, -0.9097, -1.1767],
                      [ 0.4324,  0.6304, -1.9714, -0.8295],
                      [-0.2734,  1.0736, -2.5486, -0.1443],
                      [ 0.5706,  0.7665, -0.9097, -1.1767],
                      [ 0.4324,  0.6304, -1.9714, -0.8295],
                      [-0.2734,  1.0736, -2.5486, -0.1443],
                      [ 0.5706,  0.7665, -0.9097, -1.1767],
                      [ 0.4324,  0.6304, -1.9714, -0.8295],
                      [-0.2734,  1.0736, -2.5486, -0.1443],
                      [ 0.5706,  0.7665, -0.9097, -1.1767],
                      [ 0.4324,  0.6304, -1.9714, -0.8295],
                      [-0.2734,  1.0736, -2.5486, -0.1443],
                      [ 0.5706,  0.7665, -0.9097, -1.1767],
                      [ 0.4324,  0.6304, -1.9714, -0.8295],
                      [-0.2734,  1.0736, -2.5486, -0.1443],
                      [ 0.5706,  0.7665, -0.9097, -1.1767],
                   

In [28]:
%timeit Batch.cat(batches)

166 µs ± 3.51 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

398 µs ± 7.16 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
